<a href="https://colab.research.google.com/github/Campax-org/election-chatbot/blob/main/CH_Election_chatbot_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Election chatbot

This is a data science notebook for collecting and analysing political debates, developed by Campax for the 2023 election year in Switzerland. Based on this we would like to create a chatbot, accessible via different popular messaging.

More details are in [this document](https://docs.google.com/document/d/1D9482ILL8ORyMTCPjMOUsm-EaG9Xid85EfRNO1uulJc/edit#).

In [ ]:
import requests
from pandas import Series, DataFrame